<a href="https://colab.research.google.com/github/KimJae-hee/kyakyakyakya/blob/master/%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0%ED%8A%9C%EB%8B%88%EC%9E%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.grid_search import GridSearchCV

!pip install xgboost
!pip install category_encoders
import xgboost as xgb
import category_encoders

In [ ]:
train  = pd.read_csv("/content/drive/My Drive/P-SAT/2020-2/방학 세미나/최종 데이터/cleaned_train.csv")
# test = pd.read_csv("/content/drive/My Drive/P-SAT/2020-2/방학 세미나/최종 데이터/cleaned_test.csv)

In [ ]:
train = train.drop(["ssp_id", "device_os", "device_ifa", "campaign_id", "placement_type", "advertisement_id", "age", "gender", "marry", "bid_id"], axis = 1)

In [ ]:
train_X = train.drop(['click'], axis = 1)
train_Y = train.loc[:,"click"]

In [ ]:
X_use, X_rest, Y_use, Y_rest = train_test_split(train_X, train_Y, test_size=0.8, train_size=0.2, stratify= train_Y)

In [ ]:
kfold  = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 1234)

In [ ]:
lr_list = [0.1, 0.15, 0.2]
max_list = [4, 5, 6]
min_list = [4, 5, 6]
gamma_list = [0.1, 0.2, 0.3, 0.4]
result_df = pd.DataFrame(columns=["learning_rate", "max_depth","min_chilid_weight", "gamma", "result"])
for lr in lr_list:
  for max in max_list:
    for min in min_list:
      for gamma in gamma_list:
        result_list = []
        for train_idx, test_idx in kfold.split(X_use, Y_use):
          train_fold_X, test_fold_X = X_use.iloc[train_idx,:], X_use.iloc[test_idx,:]
          train_fold_Y, test_fold_Y = Y_use.iloc[train_idx], Y_use.iloc[test_idx]

          train_x, val_x, train_y, val_y = train_test_split(train_fold_X, train_fold_Y, test_size = 0.2, train_size = 0.8, stratify = train_fold_Y, random_state = 1234)
          catboost_encoder = category_encoders.CatBoostEncoder()
          train_cbe = catboost_encoder.fit_transform(train_x, train_y)
          val_cbe = catboost_encoder.transform(val_x)

          eval_set = [(val_cbe, val_y)]


          model = xgb.XGBClassifier(random_state = 1234, n_estimators = 500, learning_rate = lr, max_depth = max, min_child_weight = min, gamma = gamma)
          model.fit(train_cbe, train_y, eval_metric = "logloss", eval_set = eval_set, early_stopping_rounds = 50, verbose = 0)
          test_cbe = catboost_encoder.transform(test_fold_X)
          pred_fold = model.predict_proba(test_cbe)

          result_tmp = log_loss(test_fold_Y, pred_fold)
          result_list.append(result_tmp)
        print(f"learning_rate : {lr}, max_depth: {max}, min_child_weight: {min} :::: {np.mean(result_list)}")
        df_tmp = pd.DataFrame([[lr, max, min, gamma, np.mean(result_list)]], columns=["learning_rate", "max_depth","min_chilid_weight", "gamma", "result"])
        result_df = result_df.append(df_tmp)

learning_rate : 0.1, max_depth: 4, min_child_weight: 4 :::: 0.21378825628489723
learning_rate : 0.1, max_depth: 4, min_child_weight: 4 :::: 0.21375164743042063
learning_rate : 0.1, max_depth: 4, min_child_weight: 4 :::: 0.21375825844429777
learning_rate : 0.1, max_depth: 4, min_child_weight: 4 :::: 0.21379658432894075
learning_rate : 0.1, max_depth: 4, min_child_weight: 5 :::: 0.21370731144518781
learning_rate : 0.1, max_depth: 4, min_child_weight: 5 :::: 0.2137939387466976
learning_rate : 0.1, max_depth: 4, min_child_weight: 5 :::: 0.2137702982885461
learning_rate : 0.1, max_depth: 4, min_child_weight: 5 :::: 0.21376902005553258
learning_rate : 0.1, max_depth: 4, min_child_weight: 6 :::: 0.2136330949392525
learning_rate : 0.1, max_depth: 4, min_child_weight: 6 :::: 0.2136929402950289
learning_rate : 0.1, max_depth: 4, min_child_weight: 6 :::: 0.21370920261777182
learning_rate : 0.1, max_depth: 4, min_child_weight: 6 :::: 0.21372353445920936
learning_rate : 0.1, max_depth: 5, min_child

In [ ]:
print(result_df.groupby("learning_rate").mean())
print(result_df.groupby("max_depth").mean())
print(result_df.groupby("min_chilid_weight").mean())
print(result_df.groupby("gamma").mean())
print(result_df.sort_values("result"))

               gamma    result
learning_rate                 
0.10            0.25  0.213836
0.15            0.25  0.213921
0.20            0.25  0.214069
           learning_rate  gamma    result
max_depth                                
4                   0.15   0.25  0.213800
5                   0.15   0.25  0.213905
6                   0.15   0.25  0.214121
                   learning_rate  gamma    result
min_chilid_weight                                
4                           0.15   0.25  0.213955
5                           0.15   0.25  0.213917
6                           0.15   0.25  0.213955
       learning_rate    result
gamma                         
0.1             0.15  0.213941
0.2             0.15  0.213940
0.3             0.15  0.213943
0.4             0.15  0.213946
    learning_rate max_depth min_chilid_weight  gamma    result
0             0.1         5                 5    0.3  0.213569
0             0.1         4                 6    0.1  0.213633
0         

In [ ]:
catboost_encoder.set_params

<bound method BaseEstimator.set_params of CatBoostEncoder(a=1,
                cols=['adset_id', 'media_id', 'media_name', 'media_bundle',
                      'publisher_id', 'publisher_name', 'device_os_version',
                      'device_model', 'device_carrier', 'device_make',
                      'device_connection_type', 'device_language',
                      'device_region', 'device_city', 'hour'],
                drop_invariant=False, handle_missing='value',
                handle_unknown='value', random_state=None, return_df=True,
                sigma=None, verbose=0)>

In [ ]:
result_df.groupby("learning_rate").mean()

,gamma,result
learning_rate,,
0.10,0.25,0.213836
0.15,0.25,0.213921
0.20,0.25,0.214069


In [ ]:
print(f"best iteration is{model.best_iteration}")
pd.DataFrame({'importance': model.feature_importances_,
              'features': train_fold_X.columns})

best iteration is223


,importance,features
0,0.036181,adset_id
1,0.692925,media_id
2,0.029522,media_name
3,0.089077,media_bundle
4,0.017191,publisher_id
5,0.032182,publisher_name
6,0.020823,device_os_version
7,0.017454,device_model
8,0.010562,device_carrier
9,0.012618,device_make


In [ ]:
train_test_split(random)

In [ ]:
# train_x, train_y, test_x, test_y
!pip install xgboost
!pip install category_encoders

import xgboost as xgb
import category_encoders

encoder = category_encoders.CatBoostEncoder()
train_cbe = encoder.fit_transform(train_x)
model = xgb.XGBClassifier()
model.fit(train_cbe, train_y)


test_cbe = encoder.transform(test_x)
model.predict_proba(test_cbe)

